# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [61]:
# download necessary NLTK data
import nltk
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
import re
import pandas as pd
from nltk.tokenize import word_tokenize
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/student/nltk_data...


True

In [62]:
# load data from database
df = pd.read_sql_table('Figure8','sqlite:///etlpipeline.db')
X = df.message.values
y = df[df.columns[4:]].values

In [63]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [64]:
X[0]

'Weather update - a cold front from Cuba that could pass over Haiti'

In [65]:
y[0]

array(['1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], dtype=object)

### 2. Write a tokenization function to process your text data

In [66]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]", "", text.lower())
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [74]:
import pandas as pd
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.multioutput import MultiOutputClassifier
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier

In [75]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [76]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [77]:
# convert predictions to a dataframe
y_pred_df = pd.DataFrame(y_pred, columns=categories)

# convert y_test to a DataFrame and select relevant columns
y_test_df = pd.DataFrame(y_test, columns=categories)
y_test_sel = y_test_df.loc[:, y_pred_df.columns]

# loop through categories and print classification report
for category in categories:
    print("Category: {}".format(category))
    print(classification_report(y_test_sel[category], y_pred_df[category]))

Category: related
             precision    recall  f1-score   support

          0       0.77      0.01      0.02      1239
          1       0.73      0.00      0.01      3967
          2       0.01      0.97      0.01        38

avg / total       0.73      0.01      0.01      5244

Category: request
             precision    recall  f1-score   support

          0       0.82      1.00      0.90      4306
          1       0.50      0.00      0.00       938

avg / total       0.76      0.82      0.74      5244

Category: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5220
          1       0.00      0.00      0.00        24

avg / total       0.99      1.00      0.99      5244

Category: aid_related
             precision    recall  f1-score   support

          0       0.58      1.00      0.73      3046
          1       0.60      0.00      0.01      2198

avg / total       0.59      0.58      0.43      5244

Category: med

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


             precision    recall  f1-score   support

          0       0.95      1.00      0.98      4994
          1       0.00      0.00      0.00       250

avg / total       0.91      0.95      0.93      5244

Category: search_and_rescue
             precision    recall  f1-score   support

          0       0.97      1.00      0.99      5095
          1       0.00      0.00      0.00       149

avg / total       0.94      0.97      0.96      5244

Category: security
             precision    recall  f1-score   support

          0       0.98      1.00      0.99      5150
          1       0.00      0.00      0.00        94

avg / total       0.96      0.98      0.97      5244

Category: military
             precision    recall  f1-score   support

          0       0.97      1.00      0.98      5083
          1       0.00      0.00      0.00       161

avg / total       0.94      0.97      0.95      5244

Category: child_alone
             precision    recall  f1-score   support

### 6. Improve your model
Use grid search to find better parameters. 

In [84]:
parameters = {
    'vect__max_df': [0.75, 1.0],
    'clf__estimator__n_estimators': [50, 100],
    'clf__estimator__min_samples_split': [2, 4],
}

In [85]:
cv = GridSearchCV(pipeline, param_grid=parameters)

In [ ]:
cv.fit(X_train, y_train)

In [ ]:
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best score: {grid_search.best_score_}")

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [23]:
y_pred = grid_search.predict(X_test)

In [24]:
# Create a DataFrame from the predictions and actual values
y_pred_df = pd.DataFrame(y_pred, columns=categories)
y_test_df = pd.DataFrame(y_test, columns=categories)

# Calculate the overall accuracy of the model
accuracy = (y_pred == y_test).mean().mean()
print(f"Overall accuracy: {accuracy:.3f}")

Overall accuracy: 0.946


In [25]:
# Iterate through the columns and print classification report for each category
for col in categories:
    print(f"{col}:")
    print(classification_report(y_test_df[col], y_pred_df[col]))

related:
             precision    recall  f1-score   support

          0       0.70      0.31      0.43      1266
          1       0.81      0.96      0.88      3938
          2       0.71      0.55      0.62        40

avg / total       0.78      0.80      0.77      5244

request:
             precision    recall  f1-score   support

          0       0.89      0.99      0.94      4349
          1       0.88      0.42      0.56       895

avg / total       0.89      0.89      0.87      5244

offer:
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5218
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      5244

aid_related:
             precision    recall  f1-score   support

          0       0.76      0.88      0.81      3113
          1       0.77      0.58      0.67      2131

avg / total       0.76      0.76      0.75      5244

medical_help:
             precision    recall  f

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


             precision    recall  f1-score   support

          0       0.98      1.00      0.99      5117
          1       1.00      0.04      0.08       127

avg / total       0.98      0.98      0.97      5244

security:
             precision    recall  f1-score   support

          0       0.98      1.00      0.99      5156
          1       0.33      0.01      0.02        88

avg / total       0.97      0.98      0.98      5244

military:
             precision    recall  f1-score   support

          0       0.97      1.00      0.99      5089
          1       0.60      0.04      0.07       155

avg / total       0.96      0.97      0.96      5244

child_alone:
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5244

avg / total       1.00      1.00      1.00      5244

water:
             precision    recall  f1-score   support

          0       0.95      1.00      0.97      4905
          1       0.87      0.26      0.40    

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [26]:
import pickle

# Save the model to a file
with open("model.pkl", "wb") as file:
    pickle.dump(grid_search, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.